# General API quickstart

In [ ]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import theano.tensor as tt

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
%config InlineBackend.figure_format='retina'
az.style.use('arviz-darkgrid')
print(f'Running on PyMC3 v{pm.__version__}')
print(f'Running on ArviZ v{az.__version__}')

## Model creation

Models in PyMC3 are centered around the `Model` class. It has references to all random variables (RVs) and computes the model logp and its gradients. Usually, you would instantiate it as part of a `with` context:

In [ ]:
with pm.Model() as model:
    # Model definition
    pass

We discuss RVs further below but let's create a simple model to explore the `Model` class.

In [ ]:
with pm.Model() as model:
    mu = pm.Normal('mu', mu=0, sigma=1)
    obs = pm.Normal('obs', mu=mu, sigma=1, observed=np.random.randn(100))

In [ ]:
model.basic_RVs

In [ ]:
model.free_RVs

In [ ]:
model.observed_RVs

In [ ]:
model.logp({'mu': 0})

It's worth highlighting the design choice we made with logp. As you can see above, `logp` is being called with arguments, so it's a method of the model instance.  More precisely, it puts together a function based on the current state of the model - or on the state given as an argument to `logp` (see example below).

For diverse reasons, we assume a `Model` instance isn't static. If you need to use `logp` in an inner loop and it needs to be static, simply use something like `logp = model.logp`. Here is an example below - note the caching effect and the speedup.

In [ ]:
%timeit model.logp({mu: 0.1})
logp = model.logp
%timeit logp({mu: 0.1})